# 🎮 꼬맨틀 게임

## 게임 소개
임베딩 기반 **코사인 유사도**로 단어의 의미적 유사도를 계산하는 게임입니다.

### 📋 게임 규칙
1. LLM이 랜덤하게 정답 단어를 생성합니다
2. **VS Code 상단 입력창**에 단어를 입력하여 추측합니다
3. **임베딩 벡터의 코사인 유사도**로 정답과의 유사도(0-100)를 계산합니다
4. '포기'를 입력하면 정답을 알려줍니다
5. 정답을 맞출 때까지 계속 도전!

### 💡 입력 방법
- 셀을 실행하면 **VS Code 화면 상단 중앙**에 입력창이 자동으로 나타납니다
- 입력창에 단어를 입력하고 Enter를 누르세요
- '기록' 또는 'ㄱ' 입력으로 시도 기록 확인 가능

### 🔬 기술 스택
- **임베딩 모델**: Azure OpenAI text-embedding-3-large
- **유사도 계산**: 코사인 유사도 (Cosine Similarity)
- **수식**: $\cos(\theta) = \frac{A \cdot B}{||A|| \times ||B||}$
- **스케일링**: Semantle 방식 (-1~1 → 0~100 선형 변환)
  - 공식: `((cosine + 1) / 2) * 100`
- **게임 언어**: 한글 단어

### 💡 게임 팁
- **한글 단어**로 입력하세요
- 카테고리 힌트를 활용하여 범위를 좁혀보세요
- 상대적 점수 변화에 집중하세요
- 90점대가 나오면 정답이 아주 가깝습니다!

## 1. 환경 설정

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from dotenv import load_dotenv
import os
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_core import CancellationToken
from autogen_core.models import UserMessage
import json
from typing import Dict, List, Tuple
from openai import AzureOpenAI
import numpy as np

# 환경 변수 로드
load_dotenv()

api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-12-01-preview")
api_key = os.getenv("AZURE_OPENAI_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
embedding_deployment = os.getenv("EMBEDDING_MODEL_NAME")

# Azure OpenAI 클라이언트 생성 (LLM용)
model_client = AzureOpenAIChatCompletionClient(
    model=deployment_name,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
    api_key=api_key,
)

# Azure OpenAI 클라이언트 생성 (임베딩용)
embedding_client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint=azure_endpoint
)

print("✅ Azure OpenAI 클라이언트 초기화 완료")

## 2. 임베딩 기반 꼬맨틀 게임 클래스

In [ ]:
class LLMKomantleGame:
    """LLM을 활용한 꼬맨틀 게임"""
    
    def __init__(self, model_client):
        self.model_client = model_client
        self.answer = None
        self.answer_vec = None  # 정답 임베딩 캐싱용
        self.attempts = []
        self.attempt_count = 0
        
    async def generate_answer(self, max_retries: int = 3) -> Tuple[str, str]:
        """LLM이 정답 단어를 생성 (실패 시 재시도)"""
        prompt = """한국어 명사 중에서 꼬맨틀 게임의 정답으로 적합한 단어 하나를 선택해주세요.
일상적인 명사가 좋습니다.

다음 카테고리 중 하나에서 선택하세요 (괄호 안은 예시이며, 예시 외 다른 단어도 선택 가능):
- 동물: 포유류, 조류, 어류, 파충류, 곤충 등 (호랑이, 참새, 거북이, 사자, 독수리, 개미, 고양이, 강아지 등)
- 과일: 계절 과일, 열대 과일 등 (포도, 수박, 망고, 사과, 바나나, 딸기, 오렌지 등)
- 음식: 한식, 양식, 중식, 일식, 디저트 등 (밥, 스파게티, 만두, 김치, 피자, 치킨, 초밥 등)
- 직업: 전문직, 서비스직, 기술직 등 (간호사, 요리사, 경찰, 의사, 선생님, 프로그래머 등)
- 장소: 실내외 공간, 자연 장소 등 (도서관, 해변, 광장, 학교, 병원, 공원, 카페 등)
- 자연: 자연 현상, 지형, 날씨 등 (구름, 계곡, 무지개, 바다, 산, 하늘, 강, 별 등)
- 감정: 긍정/부정 감정, 복합 감정 등 (기쁨, 흥분, 외로움, 행복, 슬픔, 사랑, 분노 등)
- 사물: 가전제품, 가구, 도구, 의류 등 (시계, 의자, 가방, 책, 컴퓨터, 자동차, 신발 등)

위 예시 단어들을 포함하여 각 카테고리의 다양한 단어 중에서 자유롭게 선택하세요.
매번 다른 단어를 선택하여 게임의 다양성을 높이세요.

**중요: 반드시 한글 단어로만 답변하세요.**

응답은 반드시 다음 JSON 형식으로만 답변하세요:
{
  "answer": "선택한 한글 단어 (예: 고양이, 사과)",
  "category": "카테고리 (한글)"
}
"""
        
        for attempt in range(max_retries):
            try:
                messages = [UserMessage(content=prompt, source="user")]
                response = await self.model_client.create(messages=messages, cancellation_token=CancellationToken())
                
                content = response.content
                if "```json" in content:
                    content = content.split("```json")[1].split("```")[0]
                elif "```" in content:
                    content = content.split("```")[1].split("```")[0]
                
                data = json.loads(content.strip())
                self.answer = data["answer"]
                category = data["category"]
                
                # 정답 임베딩 미리 계산 및 캐싱 (성능 최적화)
                self.answer_vec = self.get_embedding(self.answer)
                
                print(f"✅ 정답 생성 성공 (시도 {attempt + 1}/{max_retries})")
                return self.answer, category
                
            except Exception as e:
                print(f"⚠️ 정답 생성 실패 (시도 {attempt + 1}/{max_retries}): {e}")
                if attempt < max_retries - 1:
                    print("🔄 재시도 중...")
                else:
                    print("❌ 최대 재시도 횟수 초과")
                    raise Exception(f"정답 생성 실패: {max_retries}번 시도 후에도 실패했습니다.")
    
    def get_embedding(self, text: str, max_retries: int = 2) -> np.ndarray:
        """텍스트를 벡터로 변환 (재시도 로직 포함)"""
        for attempt in range(max_retries):
            try:
                response = embedding_client.embeddings.create(
                    input=text,
                    model=embedding_deployment
                )
                return np.array(response.data[0].embedding)
            except Exception as e:
                if attempt < max_retries - 1:
                    print(f"⚠️ 임베딩 생성 실패, 재시도 중... ({attempt + 1}/{max_retries})")
                    continue
                else:
                    raise Exception(f"임베딩 생성 실패: {e}")
    
    def calculate_cosine_similarity(self, vec1: np.ndarray, vec2: np.ndarray) -> float:
        """코사인 유사도 계산 (0~100 스케일)
        
        Semantle 방식: 코사인 유사도 -1~1을 0~100으로 선형 매핑
        - 이점: 모델 독립적, 일관된 감각, 투명한 해석
        - 공식: ((cosine + 1) / 2) * 100
        """
        dot_product = np.dot(vec1, vec2)
        norm1 = np.linalg.norm(vec1)
        norm2 = np.linalg.norm(vec2)
        
        if norm1 == 0 or norm2 == 0:
            return 0.0
        
        # 코사인 유사도 계산 (-1 ~ 1)
        cosine_sim = float(dot_product / (norm1 * norm2))
        cosine_sim = max(-1.0, min(1.0, cosine_sim))  # 클리핑
        
        # Semantle 방식: -1~1 → 0~100 선형 변환
        similarity = ((cosine_sim + 1.0) / 2.0) * 100.0
        
        return float(similarity)
    
    async def calculate_similarity(self, word: str) -> Tuple[float, str]:
        """임베딩 기반 코사인 유사도 계산"""
        if word == self.answer:
            return 100.0, "🎉 정답입니다!"
        
        # 캐싱된 정답 벡터 사용 (성능 최적화)
        word_vec = self.get_embedding(word)
        
        # 코사인 유사도 계산
        similarity = self.calculate_cosine_similarity(self.answer_vec, word_vec)
        
        # 재조정된 유사도 구간 (더 엄격하고 세분화)
        if similarity >= 95:
            reason = "거의 정답 수준"
        elif similarity >= 90:
            reason = "매우 밀접한 의미적 관계"
        elif similarity >= 85:
            reason = "강한 의미적 연관성"
        elif similarity >= 75:
            reason = "어느 정도 의미적 관련성"
        elif similarity >= 60:
            reason = "약한 의미적 관련성"
        elif similarity >= 40:
            reason = "거의 관련 없음"
        else:
            reason = "의미적 연관성 매우 낮음"
        
        return similarity, reason
    
    def get_feedback(self, similarity: float) -> str:
        """유사도에 따른 피드백 (재조정된 구간)"""
        if similarity >= 95:
            return "🔥🔥🔥 완전 불타오른다! 바로 코앞이에요!"
        elif similarity >= 90:
            return "🔥🔥 불타오른다! 거의 다 왔어요!"
        elif similarity >= 85:
            return "🔥 뜨겁다! 꽤 가깝습니다!"
        elif similarity >= 75:
            return "😊 좋아요! 어느 정도 관련이 있어요"
        elif similarity >= 60:
            return "🤔 음... 조금 관련이 있는 것 같아요"
        elif similarity >= 40:
            return "😕 아직 멀어요"
        else:
            return "❄️ 매우 멀어요. 완전 다른 분야네요!"
    
    async def guess(self, word: str) -> Dict:
        """단어 추측"""
        self.attempt_count += 1
        
        if word == "포기":
            return {
                "type": "give_up",
                "answer": self.answer,
                "attempts": self.attempt_count - 1
            }
        
        if word == self.answer:
            return {
                "type": "correct",
                "attempts": self.attempt_count,
                "word": word
            }
        
        similarity, reason = await self.calculate_similarity(word)
        feedback = self.get_feedback(similarity)
        
        self.attempts.append({
            "word": word,
            "similarity": similarity,
            "reason": reason,
            "rank": self.attempt_count
        })
        
        return {
            "type": "guess",
            "word": word,
            "similarity": similarity,
            "reason": reason,
            "feedback": feedback,
            "attempts": self.attempt_count
        }
    
    def show_history(self, top_n: int = 10):
        """시도 기록 보기"""
        if not self.attempts:
            print("아직 시도한 단어가 없습니다.")
            return
        
        sorted_attempts = sorted(self.attempts, key=lambda x: x['similarity'], reverse=True)
        
        print(f"\n📊 상위 {min(top_n, len(sorted_attempts))}개 시도:")
        print("="*80)
        print(f"{'순위':<4} {'단어':<12} {'유사도':<8} {'이유':<50}")
        print("="*80)
        
        for i, attempt in enumerate(sorted_attempts[:top_n], 1):
            similarity = attempt['similarity']
            bar_length = int(similarity / 5)
            bar = "█" * bar_length
            reason = attempt['reason'][:45] + "..." if len(attempt['reason']) > 45 else attempt['reason']
            print(f"{i:2d}.  {attempt['word']:<12} {similarity:5.1f} {bar:<20} {reason}")
        
        print("="*80)
        print(f"총 시도 횟수: {self.attempt_count}\n")

print("✅ 꼬맨틀 게임 클래스 정의 완료")

## 3. 게임 플레이

In [ ]:
async def play_game():
    """꼬맨틀 게임 플레이"""
    game = LLMKomantleGame(model_client)
    
    print("🎲 LLM이 정답 단어를 생성하는 중...\n")
    answer, category = await game.generate_answer()
    
    print("="*80)
    print("🎮 게임 시작!")
    print(f"📁 카테고리: {category}")
    print(f"💡 정답은 {len(answer)}글자 한글 단어입니다.")
    print("\n명령어:")
    print("  - 한글 단어 입력: 추측하기 (예: 고양이, 강아지, 사과)")
    print("  - '포기': 정답 보기 및 게임 종료")
    print("  - '기록' 또는 'ㄱ': 시도 기록 보기")
    print("="*80)
    
    game_over = False
    
    while not game_over:
        print("\n")
        word = input("💬 한글 단어를 입력하세요: ").strip()
        
        if not word:
            print("⚠️ 한글 단어를 입력해주세요!")
            continue
        
        if word in ["기록", "ㄱ"]:
            game.show_history(top_n=15)
            continue
        
        print(f"\n🎯 입력: '{word}'")
        print("⏳ 임베딩으로 유사도를 계산하는 중...\n")
        
        result = await game.guess(word)
        
        if result["type"] == "give_up":
            print("\n😢 포기하셨습니다.")
            print("="*60)
            print(f"📝 정답: {result['answer']}")
            print(f"🎯 시도 횟수: {result['attempts']}번")
            print("="*60)
            game.show_history()
            game_over = True
            
        elif result["type"] == "correct":
            print("\n🎉🎉🎉 축하합니다! 정답을 맞추셨습니다! 🎉🎉🎉")
            print("="*60)
            print(f"✅ 정답: {result['word']}")
            print(f"🎯 시도 횟수: {result['attempts']}번")
            print("="*60)
            game.show_history()
            game_over = True
            
        else:
            print("─"*80)
            print(f"📊 유사도: {result['similarity']:.1f}점")
            print(f"💭 평가: {result['reason']}")
            print(f"🎭 {result['feedback']}")
            print(f"📈 시도 횟수: {result['attempts']}번")
            print("─"*80)
            
    if game.attempts:
        avg_similarity = sum(a['similarity'] for a in game.attempts) / len(game.attempts)
        max_similarity = max(a['similarity'] for a in game.attempts)
        best_guess = max(game.attempts, key=lambda x: x['similarity'])
        
        print("\n📈 게임 통계")
        print("="*60)
        print(f"🎯 정답: {game.answer}")
        print(f"📊 총 시도 횟수: {game.attempt_count}")
        print(f"📈 평균 유사도: {avg_similarity:.2f}")
        print(f"🏆 최고 유사도: {max_similarity:.2f}")
        print(f"👍 가장 가까웠던 추측: '{best_guess['word']}' ({best_guess['similarity']:.1f}점)")
        print("="*60)

# 게임 시작
await play_game()